In [1]:
%pip install wikipedia-api
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from bs4 import BeautifulSoup
from IPython import embed
import pandas as pd
import re
import requests
import random

import wikipediaapi
import wikipedia
wiki_wiki = wikipediaapi.Wikipedia('en')

BASE_URL = "https://famous-mathematicians.com/list/"


def get_random_number():
    return random.randint(1, 5)

def delay(seconds):
    print(f"Sleeping for {seconds} second(s)")
    

def extract_html_content(target_url):
    response = requests.get(target_url)
    return response.text

def extract_target_from_value(target_url):
    
    soup = BeautifulSoup(target_url, 'html.parser')
    
    mathematicians = {}
    
    firstname = []
    for div in soup.find_all('td', class_='column-1'):
        if div.text.strip == '':
            firstname.append(' ')
        else:
            firstname.append(div.text.lower())
        
    lastname = []   
    for div in soup.find_all('td', class_='column-2'):
        lastname.append(div.text.lower())
    
    dates = []   
    for div in soup.find_all('td', class_='column-3'):
        dates.append(div.text)
        
    nationality = []    
    for div in soup.find_all('td', class_='column-4'):
        nationality.append(div.text.lower())
        
    known_for = []    
    for div in soup.find_all('td', class_='column-5'):
        if div.text.strip == '':
            known_for.append(' ')
        else:
            known_for.append(div.text.lower())

    summary_list = []
    for idx, val in enumerate(lastname):
        print(idx)
        name = '{} {}'.format(firstname[idx].strip(), lastname[idx].strip())
        page_py = wiki_wiki.page(str(name))
        r1 = re.search("\(([^)]+)\)", page_py.summary)
        summary = page_py.summary
        summary = summary.split("\n")
        summary = summary[0]
        result = page_py.summary
        if r1 is not None:
            span = r1.span()
            result = result.replace(str(result[span[0]:span[1]]), "")
        
        result = result.split(".")
        if len(result) < 0:
            summary_list.append(' ')
        else:
            summary_list.append(result[0])
    
    df = pd.DataFrame(data={'firstname': firstname, 'lastname': lastname, 
                            'dates': dates, 'nationality': nationality, 'known_for': known_for, 
                            'summary': summary_list})
    df.to_csv("mathematicians2.csv", sep=',',index=False)
    print(len(firstname))
    print(len(lastname))
    print(len(dates))
    print(len(nationality))
    print(len(known_for))
    print(len(summary_list))
    

def main():
    text = extract_html_content(BASE_URL)
    extract_target_from_value(text)
        
if __name__ == "__main__":
    main()

ConnectionError: HTTPSConnectionPool(host='famous-mathematicians.com', port=443): Max retries exceeded with url: /list/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fde3982b4c0>: Failed to establish a new connection: [Errno 60] Operation timed out'))